In [1]:
# import
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup
import pandas as pd
import duckdb
import threading
import win32com.client
from pretty_html_table import build_table
import time
from datetime import datetime

In [2]:
## Chaldal ##
def scrape_chaldal_thread(brands, lock, df_acc): 
    
    # accumulator
    df_acc_local = pd.DataFrame()

    # open window
    driver = webdriver.Chrome('chromedriver', options=[])
    driver.maximize_window()
    wait = WebDriverWait(driver, 40)

    # url
    for b in brands:
        url = "https://chaldal.com/search/" + b
        driver.get(url)
        
        # scroll
        SCROLL_PAUSE_TIME = 5
        last_height = driver.execute_script("return document.body.scrollHeight")
        while True:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(SCROLL_PAUSE_TIME)
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height: break
            last_height = new_height

        # soup
        soup_init = BeautifulSoup(driver.page_source, 'html.parser')
        soup = soup_init.find_all("div", attrs={"class": "imageWrapper"})
        
        # scrape
        skus = []
        quants = []
        prices = []
        prices_if_discounted = []
        for s in soup:
            # sku
            try: val = s.find("div", attrs={"class": "name"}).get_text()
            except: val = None
            skus.append(val)
            # quantity
            try: val = s.find("div", attrs={"class": "subText"}).get_text()
            except: val = None
            quants.append(val)
            # price
            try: val = float(s.find("div", attrs={"class": "price"}).get_text().split()[1].replace(',', ''))
            except: val = None
            prices.append(val)
            # discount
            try: val = float(s.find("div", attrs={"class": "discountedPrice"}).get_text().split()[1].replace(',', ''))
            except: val = None
            prices_if_discounted.append(val)
        
        # accumulate
        df = pd.DataFrame()
        df['sku'] = skus
        df['brand'] = b
        df['quantity'] = quants
        df['price'] = prices
        df['price_if_discounted'] = prices_if_discounted
        
        # relevant data
        qry = '''
        select *
        from
            (select *, row_number() over() pos_in_pg
            from df
            ) tbl1 
        where sku ilike ''' + "'" + b + '''%';
        '''
        df = duckdb.query(qry).df()
        rel_idx = df['pos_in_pg'].tolist()
        len_rel_idx = len(rel_idx)
        
        # description
        bnr = 3
        try: driver.find_element(By.CLASS_NAME, "important-banner")
        except: bnr = 2
        descs = []
        for i in range(0, len_rel_idx): 
            descs.append("ERROR")
            try:
                # move
                path = '//*[@id="page"]/div/div[6]/section/div/div/div/div/section/div['+str(bnr)+']/div[2]/div['+str(rel_idx[i])+']/div/div'
                elem = driver.find_element(By.XPATH, path)
                mov = ActionChains(driver).move_to_element(elem)
                mov.perform()
                # details
                path1 = '//*[@id="page"]/div/div[6]/section/div/div/div/div/section/div['+str(bnr)+']/div[2]/div['+str(rel_idx[i])+']/div/div/div[5]/span/a'
                path2 = '//*[@id="page"]/div/div[6]/section/div/div/div/div/section/div['+str(bnr)+']/div[2]/div['+str(rel_idx[i])+']/div/div/div[6]/span/a'
                try: elem = driver.find_element(By.XPATH, path1)
                except: elem = driver.find_element(By.XPATH, path2)
                elem.click()
                # content
                path = '//*[@id="page"]/div/div[6]/section/div/div/div/div/section/div['+str(bnr)+']/div[2]/div['+str(rel_idx[i])+']/div/div[2]/div/div/article/section[2]/div[5]'
                elem = driver.find_element(By.XPATH, path)
                descs[i] = elem.text.replace("\n", " ")
                # close
                path = '//*[@id="page"]/div/div[6]/section/div/div/div/div/section/div['+str(bnr)+']/div[2]/div['+str(rel_idx[i])+']/div/div[2]/div/button'
                elem = driver.find_element(By.XPATH, path)
                elem.click()
            except: pass
        # progress
        lock.acquire()
        print("Descriptions fetched for: " + b)
        lock.release()        
        
        # stock
        stocks = []
        report_times = []
        for i in rel_idx: 
            stk = 0
            try: 
                # add to bag
                path = '//*[@id="page"]/div/div[6]/section/div/div/div/div/section/div[2]/div[2]/div['+str(i)+']/div/section'
                elem = driver.find_element(By.XPATH, path)
                clks = 1
                while(1): 
                    mov = ActionChains(driver).move_to_element(wait.until(EC.element_to_be_clickable(elem)))
                    for j in range (0, clks): mov.click().perform()
                    # check unavailability
                    path = '//*[@id="page"]/div/div[6]/section/div/div/div/div/section/div[2]/div[2]/div['+str(i)+']/div/section/p'
                    try: 
                        # read bag
                        stk = int(elem.text.split()[0].replace(",", ""))
                        elem = driver.find_element(By.XPATH, path)
                        break
                    except: pass
                    # add more to bag
                    path = '//*[@id="page"]/div/div[6]/section/div/div/div/div/section/div[2]/div[2]/div['+str(i)+']/div/div[2]/div'
                    try: elem = driver.find_element(By.XPATH, path)
                    except: break
                    clks = 100
            except: stk = None
            # report bag
            lock.acquire()
            print("Stock for " + skus[i-1] + " " + quants[i-1] + ": " + str(stk))
            lock.release()
            stocks.append(stk)
            report_times.append(time.strftime('%Y-%m-%d %H:%M:%S'))
    
        # accumulate
        df['stock'] = stocks
        df['description'] = descs
        df['report_time'] = report_times
        df_acc_local = df_acc_local.append(df)
        
    # return
    df_acc[df_acc_local.columns.tolist()] = df_acc_local
        
    # close window
    driver.close()

# caller
def scrape_chaldal():
    
    # accumulators
    start_time = time.time()
    brands = ['Boost', 'Clear', 'Simple', 'Pepsodent', 'Brylcreem', 'Bru', 'St. Ives', 'Horlicks', 'Sunsilk', 'Lux', 'Pond', 'Closeup', 'Cif', 'Dove', 'Maltova', 'Domex', 'Clinic', 'Tresemm', 'GlucoMax', 'Knorr', 'Glow & Lovely', 'Glow & Handsome', 'Wheel', 'Axe', 'Pureit', 'Lifebuoy', 'Surf Excel', 'Vaseline', 'Vim', 'Rin']
    thread_count = 3
    brands_chunks = [brands[i::thread_count] for i in range(thread_count)]
    dfs_acc = [pd.DataFrame() for i in range(thread_count)]
    lock = threading.Lock()

    # threads
    threads = []
    for i in range(0, thread_count):
        threads.append(threading.Thread(target=scrape_chaldal_thread, args=(brands_chunks[i], lock, dfs_acc[i])))
    for t in threads:
        t.start()
    for t in threads:
        t.join()

    # csv
    df_acc = pd.DataFrame()
    for i in range(0, thread_count):
        df_acc = df_acc.append(dfs_acc[i])
    folder = r"C:\\Users\\Shithi.Maitra\\Unilever Codes\\Scraping Scripts\\Chaldal Stocks\\"
    filename = folder + "chaldal_unilever_stocks_data_" + datetime.today().strftime('%Y-%m-%d') + ".csv"
    df_acc.to_csv(filename, index=False)

    # analysis
    qry = '''
    select * 
    from 
        (select 
            'Chaldal' platform, 
            count(sku) "SKUs", 
            count(case when stock=0 then sku else null end) "SKUs out of stock", 
            count(case when stock is null then sku else null end) "SKUs failed to scrape stock",
            ''' + str(len(brands)) + '''-count(distinct brand) "brands failed to scrape stock",
            count(case when length(description)=0 then sku else null end) "SKUs not described",
            min(strptime(report_time, '%Y-%m-%d %H:%M:%S')) "stocking start time",
            right(age(max(strptime(report_time, '%Y-%m-%d %H:%M:%S')), min(strptime(report_time, '%Y-%m-%d %H:%M:%S'))), 8) "time to scrape stocks"
        from df_acc
        ) tbl1,

        (select 
            brand "longest stock to scrape", 
            right(age(max(strptime(report_time, '%Y-%m-%d %H:%M:%S')), min(strptime(report_time, '%Y-%m-%d %H:%M:%S'))), 8) "longest time to scrape stock"
        from df_acc
        group by 1
        order by 2 desc
        limit 1
        ) tbl2; 
    '''
    res_df = duckdb.query(qry).df()
    
    # stats
    print("\nTotal SKUs found: " + str(df_acc.shape[0]))
    elapsed_time = str(round((time.time() - start_time) / 60.00, 2))
    print("Elapsed time to run script (mins): " + elapsed_time)
    
    return res_df

In [3]:
# main/summary
smry_df = scrape_chaldal()
smry_df

Descriptions fetched for: Boost
Descriptions fetched for: Simple
Stock for Boost Chocolate Drink Jar 400 gm: 21
Stock for Simple Rich Moisture Sheet Mask 23 ml: 8
Stock for Simple De-stress, Rich Moisture and Hydrating Sheet Mask (3x23ml) Combo Pack 3 pcs: 2
Stock for Simple Soothing Facial Toner 200 ml: 0
Stock for Simple Micellar Facial Wash 150 ml: 0
Stock for Simple Refreshing Facial Wash 150 ml: 0
Stock for Simple Water Boost Hydrating Sheet Mask 23 ml: 0
Descriptions fetched for: Clear
Stock for Boost Iced Coffee Caffe Latte 250 ml: 50
Stock for Clear Men Shampoo Deep Cleanse 170 ml: 16
Stock for Clear Men Menthol Anti Dandruff Shampoo 315 ml: 6
Stock for Clear Shampoo Anti Hairfall Anti Dandruff 330 ml: 1
Stock for Clear Shampoo Anti Hairfall Anti Dandruff 170 ml: 3
Descriptions fetched for: Bru
Stock for Bru Pure Coffee 100 gm: 0
Stock for Bru Original Coffee Jar 200 gm: 0
Stock for Bru Pure Instant Coffee Jar 100 gm: 0
Stock for Bru Original Instant Coffee 100 gm: 0
Stock for 

Stock for Dove Beauty Bar Pink 135 gm: 87
Stock for Lux Body Wash French Rose And Almond Oil (Free Loofah) 245 ml: 80
Stock for Dove Beauty Bar White 90 gm: 45
Stock for Tresemmé Shampoo Colour Revitalise (Free Stylish Clutch Bag) 580 ml: 94
Stock for Lux Handwash Rose and Almond Oil Pump 200 ml: 20
Stock for Lux Body Wash Magical Orchid 500 ml: 0
Stock for Lux Body Wash Bright Camellia 500 ml: 0
Stock for Tresemmé Shampoo Keratin Smooth 340 ml: 18
Stock for Dove Shampoo Healthy Grow 170 ml: 26
Stock for Tresemmé Shampoo Keratin Smooth 185 ml: 11
Stock for Dove Rich Nourishment Body Cream 150 ml: 2
Stock for Tresemmé Shampoo Hair Fall Defense (Free Keratin Smooth Conditioner 50 ml) 580 ml: 1
Stock for Dove Beauty Cream 150 ml: 13
Stock for Tresemmé Conditioner Hair Fall Defense 190 ml: 15
Stock for Dove Silky Cream 300 ml: 2
Stock for Dove Original Moisturising Cream 50 ml: 1
Descriptions fetched for: Cif
Stock for Dove Conditioner Intense Repair 175 ml: 16
Stock for Tresemmé Condition

Stock for Surf Excel Matic Liquid Detergent Top Load 1020 ml: 0
Stock for Surf Excel Matic Liquid Detergent Top Load 500 ml: 0
Stock for Surf Excel Matic Liquid Detergent Front Load 500 ml: 0
Stock for Wheel Washing Laundry Bar 125 gm: 1503
Descriptions fetched for: Rin
Stock for Wheel Washing Powder 2 in 1 Clean & Fresh 1 kg: 2613
Stock for Rin Washing Powder Power Bright 500 gm: 874
Stock for Rin Washing Powder Power Bright 1 kg: 82
Stock for Wheel Washing Powder 2in1 Clean & Fresh 500 gm: 1357
Descriptions fetched for: Lifebuoy
Stock for Lifebuoy Handwash Total Refill 170 ml: 232
Stock for Lifebuoy Handwash Lemon Fresh Refill 170 ml: 188
Stock for Lifebuoy Soap Bar Lemon Fresh 100 gm: 105
Stock for Lifebuoy Soap Bar Care 100 gm: 113
Stock for Lifebuoy Vitamins Included Lemon Fresh Handwash 1 ltr: 84
Stock for Lifebuoy Handwash Care Refill 170 ml: 95
Stock for Lifebuoy Handwash Care 1 ltr: 22
Stock for Lifebuoy Handwash Total Pump 200 ml: 56
Stock for Lifebuoy Handwash Total 1 ltr: 5

,platform,SKUs,SKUs out of stock,SKUs failed to scrape stock,brands failed to scrape stock,SKUs not described,stocking start time,time to scrape stocks,longest stock to scrape,longest time to scrape stock
0,Chaldal,303,92,0,1,145,2023-03-02 11:48:20,01:09:27,Rin,00:27:44


In [4]:
# email
ol = win32com.client.Dispatch("outlook.application")
olmailitem = 0x0
newmail = ol.CreateItem(olmailitem)

# subject, recipients
newmail.Subject = 'Chaldal Stocks ' + time.strftime('%d-%b-%y')
# newmail.To = 'shithi.maitra@unilever.com'
newmail.To = 'mehedi.asif@unilever.com'
# newmail.CC = 'mehedi.asif@unilever.com; zakeea.husain@unilever.com; rakaanjum.unilever@gmail.com; nazmussajid.ubl@gmail.com'

# body
newmail.HTMLbody = f'''
Dear concern,<br><br>
Today's <i>Chaldal</i> stocks for Unilever SKUs have been scraped. A brief statistics of the process is given below:
''' + build_table(smry_df, 'blue_light') + '''
Please find the data attached. Note that, this email was auto generated at ''' + time.strftime('%d-%b-%y, %I:%M %p') + ''' using <i>win32com</i>.<br><br>
Thanks,<br>
Shithi Maitra<br>
Asst. Manager, Cust. Service Excellence<br>
Unilever BD Ltd.<br>
'''

# attachment(s) 
folder = r"C:\\Users\\Shithi.Maitra\\Unilever Codes\\Scraping Scripts\\Chaldal Stocks\\"
filename = folder + "chaldal_unilever_stocks_data_" + datetime.today().strftime('%Y-%m-%d') + ".csv"
newmail.Attachments.Add(filename)

# display, send
newmail.Display()
newmail.Send()